In [25]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import MatrixBasedConvolution as mbc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Convolution parameters

In [26]:
inputs_shape_1d=(1,20)
inputs_shape_2d=(1,5,5,1)
ARGS={"kernel_size":3,
"stride":1,
"padding":'same',
"use_phi":True,
"activation":None,
"use_lambda_out":False,
"use_lambda_in":False}

## Convalution 1D and 2D outputs

In [27]:
print("1D Convolution building...")
con1d=mbc.matrix_conv_1d(**ARGS)
con1d.build(inputs_shape_1d)
print("2D Convolution building...")
con2d=mbc.matrix_conv_2d(**ARGS)
con2d.build(inputs_shape_2d)

1D Convolution building...
2D Convolution building...


In [28]:
print("inputs 1D generation...")
inputs_1d=tf.random.uniform(shape=inputs_shape_1d,minval=0,maxval=1)
print("inputs 2D generation...")
inputs_2d=tf.random.uniform(shape=inputs_shape_2d,minval=0,maxval=1)


inputs 1D generation...
inputs 2D generation...


In [29]:
ouputs_mbc_1d=con1d.conv_jit(inputs_1d).numpy()
print(ouputs_mbc_1d.shape)
ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()[0,:,:,0]
print(ouputs_mbc_2d.shape)

(1, 20)
(5, 5)


## Comparison between standard conv1d/conv2d and matrix based conv1d/conv2d

In [30]:
kernel=tf.reshape(con2d.kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size')))
kernel.shape

TensorShape([3, 3])

In [31]:
ouputs_classical=mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())
ouputs_classical.shape


(5, 5)

In [32]:
ouputs_mbc_2d

array([[ 0.19764873, -0.23331879, -0.4850127 , -0.43570155, -0.77143717],
       [ 0.4793913 ,  0.15222931,  0.71863467,  0.70394063,  0.44844094],
       [ 0.6343563 ,  0.4520318 ,  0.28845876,  0.23038262, -0.71837115],
       [ 0.26830578,  0.20875445,  0.13688466,  0.13685319,  0.08429235],
       [ 0.2162105 ,  0.29226077,  0.6303144 ,  0.4999283 ,  0.20011243]],
      dtype=float32)

In [33]:
ouputs_classical

array([[ 0.19764873, -0.23331876, -0.48501268, -0.43570152, -0.7714371 ],
       [ 0.4793913 ,  0.15222928,  0.71863467,  0.7039407 ,  0.4484409 ],
       [ 0.63435626,  0.45203173,  0.28845876,  0.23038268, -0.71837115],
       [ 0.26830578,  0.20875444,  0.13688466,  0.13685317,  0.08429234],
       [ 0.21621053,  0.29226074,  0.6303144 ,  0.49992836,  0.20011243]],
      dtype=float32)

In [34]:
if np.allclose(ouputs_classical,ouputs_mbc_2d,atol=1e-16):
    print("The outputs are the same")


The outputs are the same


## Timer

In [35]:
%timeit ouputs_mbc_2d=con2d.conv_jit(inputs_2d).numpy()


144 μs ± 1.69 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [36]:
%timeit mbc.classical_convolution(img=inputs_2d[0,:,:,0],kernel=kernel.numpy())

26.3 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
conv_layer = tf.keras.layers.Conv2D(
    filters=1,           
    kernel_size=(ARGS.get('kernel_size'), ARGS.get('kernel_size')),     
    strides=(ARGS.get('stride'), ARGS.get('stride')),  
    activation=ARGS.get('activation'),      
    padding=ARGS.get('padding'),        
    use_bias=False          
)
conv_layer.build(input_shape=inputs_2d.shape)
conv_layer.set_weights([tf.reshape(kernel,shape=(ARGS.get('kernel_size'),ARGS.get('kernel_size'),1,1)).numpy()])


In [38]:
%timeit conv_layer(inputs_2d)

488 μs ± 14.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
